<a href="https://colab.research.google.com/github/Vinicius-Tavares-Silva/Job-AI-Match/blob/main/Job_AI_Match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação dos módulos (Caso necessário)

In [ ]:
!pip install pdfx
!pip install requests==2.22.0 beautifulsoup4==4.8.1

Job AI Match: Analise seu curriculo com sua vaga dos sonhos, entenda seus pontos fortes e a melhorar!

In [23]:
# Importação dos módulos
import hashlib
import string
import requests
import textwrap
import pdfx
from pathlib import Path
from bs4 import BeautifulSoup
from IPython.display import display
from IPython.display import Markdown
from google.colab import files
from google.colab import userdata
import google.generativeai as genai

# Uso do secret do Colab
api_key = userdata.get('API_KEY')
genai.configure(api_key=api_key)

# Setup do modelo
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]
system_instruction = "Voce é um recrutador especializado em triar curriculos, deve estruturar sua resposta sempre com 4 tópicos: Pontos Fortes, Pontos a Melhorar, Recomendações, Conclusão. Caso não identifique o curriculo ou os requisitos da vaga deve alertar o usuario"
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

# Definição dos metodos que serão utilizados

# Responsável por ler o arquivo pdf
def file_reader(file_name):
  try:
    pdf = pdfx.PDFx(file_name)
    return pdf.get_text()
  except Exception as e:
    print(f"Erro: Arquivo '{file_name}' fora do formato PDF")

# Faz o upload do arquivo
def file_upload():
  try:
    uploaded = files.upload()
    return list(uploaded.keys())[0]
  except Exception as e:
    print(f"Erro: Arquivo inválido")

# Formata texto pra markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Obtem as informações da vaga atraves de um web scraping
def get_job_info(job_url):
  if len(job_url) == 0:
    return None
  try:
    html_text = requests.get(job_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    html_plain_text = soup.get_text().translate({ord(c): None for c in string.whitespace})
    return html_plain_text
  except Exception as e:
    print(f"Erro: Não foi possivel obter a vaga")
    return None

# Inicio da execução

print("Faça o upload do seu Currículo! (formato .pdf)")
print("\n")
# Campo para upload do arquivo
file_name = file_upload()
if file_name != None:
  file_content = file_reader(file_name)
  if file_content != None:
    # Instância do modelo
    recruiter = model.start_chat(history=[])
    # Envio do currículo para o modelo
    print("\nRecebendo currículo...")
    recruiter.send_message("Curriculo: " + file_content)

    # Loop que permite a analise de multiplas vagas
    job_info = 'start'
    while job_info != None:
      # Input do link da vaga pelo usúario (Para maiores informações consulte o readme)
      # exemplo de vaga LinkedIn (https://www.linkedin.com/jobs/view/3880517623)
      print("\nCole a URL da vaga a qual pretende aplicar (LinkedIn, Gupy...) ")
      print("Para finalizar aperte 'ENTER' com o campo vazio ")
      prompt = input("URL: ")
      job_info = get_job_info(prompt)
      if job_info != None:
        # Resposta do modelo com o Match da vaga
        print("\nAnalisando Match com a vaga...")
        response = recruiter.send_message("Com base no curriculo enviado, quais são os pontos fortes e quais os pontos a se melhorar em relação a vaga a seguir: " + job_info)
        display(to_markdown(f'{response.text}'))

    print("\nFim das análises. Volte sempre!")

Faça o upload do seu Currículo! (formato .pdf)




Saving Profile_linkedin.pdf to Profile_linkedin (9).pdf

Recebendo currículo...

Cole a URL da vaga a qual pretende aplicar (LinkedIn, Gupy...) 
Para finalizar aperte 'ENTER' com o campo vazio 
URL: https://www.linkedin.com/jobs/view/3880517623

Analisando Match com a vaga...


> ## Análise de Currículo para Vaga de Software Engineer II, Front-End - Uber Flash and Direct
> 
> **Pontos Fortes:**
> 
> * **Formação em Engenharia:** A formação em Engenharia Mecatrônica demonstra capacidade analítica, de resolução de problemas e de lidar com sistemas complexos, qualidades valorizadas em engenharia de software.
> * **Experiência com projetos de software:** A atuação como Analista de Projetos no Itaú Unibanco, com gestão de equipes de desenvolvimento, suporte a sistemas e mapeamento de processos, é relevante para a vaga, que exige colaboração com equipes e conhecimento do ciclo de vida de desenvolvimento.
> * **Aprendizado contínuo:** A formação em desenvolvimento web pela Trybe demonstra o interesse de Vinícius em se atualizar e adquirir novas habilidades tecnológicas. 
> 
> **Pontos a Melhorar:**
> 
> * **Experiência específica em front-end:** A vaga exige experiência com tecnologias front-end como React, JavaScript e CSS. O currículo deve destacar projetos ou experiências que demonstrem essas habilidades, mesmo que sejam projetos pessoais ou acadêmicos.
> * **Conhecimento de sistemas distribuídos e APIs:** A descrição da vaga menciona a importância de conhecimento em sistemas distribuídos e design de APIs.  Vinícius deve destacar se possui experiência ou conhecimento nessas áreas. 
> * **Familiaridade com metodologias ágeis:**  Empresas de tecnologia como a Uber costumam utilizar metodologias ágeis de desenvolvimento. Seria interessante mencionar se Vinícius possui experiência com Scrum, Kanban ou outras metodologias.
> 
> **Recomendações:**
> 
> * **Adaptar o currículo à vaga:** Revisar o currículo e destacar as habilidades e experiências mais relevantes para a posição de Software Engineer II, Front-End, com foco em React, JavaScript, CSS e sistemas distribuídos. 
> * **Quantificar resultados:** Ao descrever experiências anteriores, quantificar os resultados obtidos, demonstrando o impacto de seu trabalho. 
> * **Incluir projetos pessoais:**  Se Vinícius possui projetos pessoais que utilizam as tecnologias exigidas pela vaga, é importante incluí-los no currículo ou portfólio online.
> * **Preparar-se para perguntas técnicas:** A entrevista para essa vaga provavelmente envolverá perguntas técnicas sobre React, JavaScript e CSS. Vinícius deve estar preparado para discutir seu conhecimento nessas áreas.
> 
> **Conclusão:**
> 
> Vinícius possui uma base sólida e habilidades relevantes para a vaga, mas precisa adaptar seu currículo para destacar melhor suas competências em desenvolvimento front-end e familiaridade com as tecnologias e metodologias utilizadas pela Uber. Ao fazer esses ajustes e se preparar para a entrevista técnica, ele aumenta suas chances de sucesso no processo seletivo.



Cole a URL da vaga a qual pretende aplicar (LinkedIn, Gupy...) 
Para finalizar aperte 'ENTER' com o campo vazio 
URL: 

Fim das análises. Volte sempre!
